In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, callback
from optuna.integration import XGBoostPruningCallback
import mlflow
from utils.metrics import Metrics_df,print_metrics, precision, recall, f1


In [2]:


import pandas as pd
from sklearn.metrics import roc_auc_score
from Config import config
import warnings
warnings.filterwarnings("ignore")
import optuna  # pip install optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import gc
import pickle
import tempfile

In [3]:
from lightgbm.basic import LightGBMError
from lightgbm import LGBMClassifier, early_stopping, log_evaluation, Booster
from sklearn.metrics import roc_auc_score
import os

# Loading data and group split

In [4]:
# df_pairs = pd.read_csv("/workspace/Training_data/training/Full_training_data.csv")
# df_pairs = df_pairs.sample(frac=1).reset_index(drop=True)
# kf = GroupKFold(n_splits=5)
# for i, (trn_idx, val_idx) in enumerate(kf.split(df_pairs, df_pairs["placeId1"], df_pairs["placeId1"])):
#     df_pairs.loc[val_idx, "fold"] = i
# df_pairs["fold"].value_counts()
# df_pairs.to_csv("/workspace/Training_data/training/Full_training_data_fold.csv",index=None)

# Loading data

In [5]:
df_pairs = pd.read_csv("/workspace/Training_data/training/Full_training_data_fold.csv")

In [6]:
features = ['similarity', 'haversine', 'name_davies',
       'name_leven', 'name_dice', 'name_jaro', 'name_set_ratio',
       'street_davies', 'street_leven', 'street_jaro', 'email_davies',
       'email_leven', 'email_jaro', 'url_davies', 'url_leven', 'url_jaro',
       'brands_davies', 'brand_leven', 'brand_jaro', 'phone_lcs',
       'subcat_WRatio', 'subcat_ratio', 'subcat_token_set_ratio',
       'Is_direction_match_0', 'Is_direction_match_1', 'Is_direction_match_2',
       'Is_house_match_0', 'Is_house_match_1', 'Is_house_match_2',
       'Is_category_match_0', 'Is_category_match_1', 'Is_subcategory_match_0',
       'Is_subcategory_match_1', 'Is_subcategory_match_2', 'Is_brand_match_0',
       'Is_brand_match_1', 'Is_brand_match_2', 'Is_brand_match_3',
       'Is_related_cat_0', 'Is_related_cat_1', 'Is_name_number_match_0',
       'Is_name_number_match_1', 'Is_name_number_match_2',
       'Is_name_number_match_3', 'is_phone_match_1', 'is_phone_match_2',
       'is_phone_match_3', 'is_phone_match_4', 'Is_email_match_0',
       'Is_email_match_1', 'Is_email_match_2', 'Is_url_match_0',
       'Is_url_match_1', 'Is_url_match_2']

In [7]:
X_train = df_pairs[df_pairs["fold"]!=4]

In [8]:
X_test = df_pairs[df_pairs["fold"]==4]

In [9]:
def objective(trial, X_train,X_test,Y_train,Y_test):
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 20),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.5, 1),
        "subsample": trial.suggest_loguniform("subsample", 0.4, 1),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 1, 300),
    }
    
    
    # cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)
    cback = [XGBoostPruningCallback(trial, "validation_0-auc")]
    
    model = XGBClassifier(
        **params,
        n_estimators=10000,
        learning_rate=0.1,
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="gpu_hist",
        use_label_encoder=False,
        random_state=42,
        enable_categorical=True,
    )

        
        
    model.fit(X_train,Y_train,eval_set=[(X_test, Y_test)],verbose=100,early_stopping_rounds=20,callbacks=cback)
        
    with tempfile.TemporaryFile() as dump_file:        
        pickle.dump(model , dump_file)
        dump_file.seek(0)
        model = pickle.load(dump_file)
    preds = model.predict_proba(X_test)[:, 1]
    del trial
    return roc_auc_score(Y_test, preds)

In [10]:
date_log = datetime.now().strftime("%Y_%m_%d_%H_%M")
study = optuna.create_study(direction="maximize", study_name=f"parameter_tuning_{config.country}_{date_log}")
func = lambda trial: objective(trial, X_train[features],X_test[features],X_train["duplicate_flag"],X_test["duplicate_flag"])
study.optimize(func, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2022-12-09 19:09:35,546] A new study created in memory with name: parameter_tuning_NZL_2022_12_09_19_09


[0]	validation_0-auc:0.99730
[100]	validation_0-auc:0.99978
[200]	validation_0-auc:0.99980
[300]	validation_0-auc:0.99981
[400]	validation_0-auc:0.99982
[500]	validation_0-auc:0.99982
[539]	validation_0-auc:0.99982


[I 2022-12-09 19:10:23,185] Trial 0 finished with value: 0.9998229974908446 and parameters: {'max_depth': 14, 'colsample_bytree': 0.9186136272179423, 'subsample': 0.6850176145347159, 'alpha': 0.3584883757182529, 'lambda': 0.0001760402607513321, 'min_child_weight': 126.44241820052189}. Best is trial 0 with value: 0.9998229974908446.


[0]	validation_0-auc:0.99676
[100]	validation_0-auc:0.99981
[200]	validation_0-auc:0.99984
[300]	validation_0-auc:0.99985
[400]	validation_0-auc:0.99985
[500]	validation_0-auc:0.99985
[600]	validation_0-auc:0.99986
[636]	validation_0-auc:0.99986


[I 2022-12-09 19:11:15,648] Trial 1 finished with value: 0.9998566930332051 and parameters: {'max_depth': 9, 'colsample_bytree': 0.5269845457413087, 'subsample': 0.9346247586259055, 'alpha': 0.012062027528091367, 'lambda': 0.0085505761406352, 'min_child_weight': 94.37961567971094}. Best is trial 1 with value: 0.9998566930332051.


...

[0]	validation_0-auc:0.99920
[100]	validation_0-auc:0.99991
[168]	validation_0-auc:0.99992


[I 2022-12-09 19:35:58,271] Trial 48 finished with value: 0.9999188344917571 and parameters: {'max_depth': 19, 'colsample_bytree': 0.6409458191324515, 'subsample': 0.9486584299767553, 'alpha': 0.04905574988631387, 'lambda': 0.001107445401896605, 'min_child_weight': 2.1698296076809367}. Best is trial 33 with value: 0.9999220485920526.


[0]	validation_0-auc:0.99839


[I 2022-12-09 19:36:01,195] Trial 49 pruned. Trial was pruned at iteration 0.


Number of finished trials: 50
Best trial: {'max_depth': 15, 'colsample_bytree': 0.5891797914157996, 'subsample': 0.9915131532117468, 'alpha': 0.01601488896353689, 'lambda': 6.471320277059568e-08, 'min_child_weight': 4.639909639888095}


In [14]:
study.trials_dataframe().to_csv("/workspace/Xgboost_trials.csv",index=None)